In [23]:
#importing 
#importing 
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator as data_augment
from keras.models import Model,Sequential
from keras.layers import Input,Conv2D,MaxPooling2D,Dropout,Flatten,Dense,GlobalAveragePooling2D,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras import layers as layers
from keras.layers.merge import concatenate
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.densenet import DenseNet169
#data augmetation 
data_generate_training = data_augment (rescale=1./255, 
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,)

data_generate_test = data_augment(rescale = 1./255)

#data preprocessing and augmentation
traind = data_generate_training.flow_from_directory("../input/skin-cancer-malignant-vs-benign/train",
                                          target_size = (224, 244),
                                          class_mode = 'binary',
                                          seed = 123,
                                          batch_size = 32,
                                          shuffle = True)

testd = data_generate_training.flow_from_directory("../input/skin-cancer-malignant-vs-benign/test",
                                          target_size = (224, 244),
                                          class_mode = 'binary',
                                          seed = 123,
                                          batch_size = 32,
                                          shuffle = True)

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [24]:
#Building pre-trained model
mobilenet_base = MobileNetV2(weights = 'imagenet', input_shape = (224, 224, 3), include_top = False)
densenet_base = DenseNet169(weights = 'imagenet', input_shape = (224, 224, 3), include_top = False)

for layer in mobilenet_base.layers:
    layers.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False
    
input_shape = (224,224,3)
input_layer = Input(shape = (224, 224, 3))
#first model
mobilenet_base = MobileNetV2(weights = 'imagenet',input_shape = input_shape,include_top = False)
densenet_base = DenseNet169(weights = 'imagenet', input_shape = input_shape,include_top = False)
for layer in mobilenet_base.layers:
    layer.trainable =  False
for layer in densenet_base.layers:
    layer.trainable = False
    
model_mobilenet = mobilenet_base(input_layer)
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)
model_densenet = densenet_base(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)

merged = tf.keras.layers.Concatenate()([output_mobilenet, output_densenet])

In [25]:
#Building Model
x = BatchNormalization()(merged)
x = Dense(256,activation = 'relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = 'sigmoid')(x)
CNNmodel = tf.keras.models.Model(inputs = input_layer, outputs = x)


In [26]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./stacked_model.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

In [ ]:
nb_train_samples = 2637 # number of training-samples
nb_validation_samples = 660 # number of validation-samples
nb_test_samples = 624 # number of training-samples
epochs = 20  
batch_size  = 16

optm = tf.keras.optimizers.Adam(learning_rate=0.0001)
CNNmodel.compile(optimizer = optm,
              loss="binary_crossentropy",
              metrics=['accuracy'])

history = CNNmodel.fit(traind,
                      epochs = 20, validation_data = testd)

Epoch 1/20
83/83 [==============================] - 381s 4s/step - loss: 0.4661 - accuracy: 0.7952 - val_loss: 0.3401 - val_accuracy: 0.8470
Epoch 2/20
83/83 [==============================] - 361s 4s/step - loss: 0.4388 - accuracy: 0.8096 - val_loss: 0.3183 - val_accuracy: 0.8652
Epoch 3/20
83/83 [==============================] - 360s 4s/step - loss: 0.4348 - accuracy: 0.8047 - val_loss: 0.3136 - val_accuracy: 0.8636
Epoch 4/20
83/83 [==============================] - 363s 4s/step - loss: 0.3949 - accuracy: 0.8161 - val_loss: 0.3140 - val_accuracy: 0.8591
Epoch 5/20
83/83 [==============================] - 362s 4s/step - loss: 0.3861 - accuracy: 0.8350 - val_loss: 0.3025 - val_accuracy: 0.8697
Epoch 6/20
83/83 [==============================] - 362s 4s/step - loss: 0.3709 - accuracy: 0.8350 - val_loss: 0.2956 - val_accuracy: 0.8591
Epoch 7/20
83/83 [==============================] - 364s 4s/step - loss: 0.3657 - accuracy: 0.8366 - val_loss: 0.3014 - val_accuracy: 0.8682
Epoch 8/20
83